In [1]:
# Povm

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
start_all = time.time()

In [4]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
tester_objects = [state_x0, state_y0, state_z0, state_z1]

In [5]:
a0, a1, a2, a3 = 1, 1 / np.sqrt(2), 0, 1 / np.sqrt(2)

m1 = (1 / np.sqrt(2)) * np.array([a0, a1, a2, a3])
m2 = (1 / np.sqrt(2)) * np.array([2 - a0, -a1, -a2, -a3])

true_object = Povm(vecs=[m1, m2], c_sys=c_sys)

In [6]:
num_data = [100, 1000]
n_rep = 1000
measurement_n = len(true_object.vecs)  # 測定値の数

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

seed = 777
qtomography_list = [
    StandardPovmt(
        tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed
    ),
    StandardPovmt(
        tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed
    ),
    StandardPovmt(
        tester_objects, measurement_n, on_para_eq_constraint=True, seed=seed
    ),
    StandardPovmt(
        tester_objects, measurement_n, on_para_eq_constraint=False, seed=seed
    ),
]
para_list = [True, False, True, False]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
]

estimation_results_list = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    
    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")

  0%|          | 5/1000 [00:00<00:22, 44.77it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardPovmt
Estimator: LinearEstimator


  1%|          | 8/1000 [00:00<00:13, 73.68it/s]

elapsed_time:0.29594791332880654[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardPovmt
Estimator: LinearEstimator


  0%|          | 0/1000 [00:00<?, ?it/s]/Users/tomoko/project/rcast/workspace/quara/quara/objects/povm.py:260: ComplexWarning: Casting complex values to real discards the imaginary part
  new_vec = np.array(new_vec, dtype=np.float64)


elapsed_time:0.31678319772084557[min]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardPovmt
Estimator: ProjectedLinearEstimator


  0%|          | 0/1000 [00:00<?, ?it/s]

elapsed_time:4.769564080238342[min]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardPovmt
Estimator: ProjectedLinearEstimator


100%|██████████| 1000/1000 [04:53<00:00,  3.41it/s]

elapsed_time:4.884150437513987[min]



In [7]:
report.export_report("sample_povmt.pdf",
    estimation_results_list,
    case_name_list,
    qtomography_list,
    para_list,
    estimator_list,
    true_object,
    tester_objects,
    seed=seed,
)

​Generating table of experimental conditions ...
Generating case list ...


100%|██████████| 1000/1000 [00:00<00:00, 124161.63it/s]

​​Generating MSE of empirical distributions blocks ...


​​Generating consictency test blocks ...
​Generating a graph for MSE ...


/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars

/Users/tomoko/project/rcast/workspace/quara/quara/objects/povm.py:260: ComplexWarning:

Casting complex values to real discards the imaginary part



​​Generating physicality violation test blocks ...


  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 62.52it/s]

2it [00:00, 51.97it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 32.41it/s]

2it [00:00, 55.03it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 56.08it/s]

2it [00:00, 47.04it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 48.96it/s]

2it [00:00, 50.29it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 45.84it/s]

2it [00:00, 51.89it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 56.11it/s]

2it [00:00, 48.58it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 56.78it/s]

2it [00:00, 47.56it/s]
  0%|          | 0/2 [00:00<?, ?it/s]
2it [00:00, 38.13it/s]

2it [00:00, 67.56it/s]
100%|██████████| 1000/1000 [00:00<00:00, 1724.15it/s]


Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (sample_povmt.pdf)
